Script to post-process high-resolution WRF model output.

Major tasks include computing the following for selected variables:
  1. domain-averages to produce time series
  2. vertical integrals
  3. pressure-level vertical interpolation

James Ruppert  
25 Nov 2024

In [11]:
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, ALL_TIMES#, vinterp
import xarray as xr
from read_wrf_piccolo import *
from post_proc_functions import *
import os

In [12]:
do_2d_special = True
do_3d_vars = False
do_3d_special = False

In [16]:
########################################################
# Directories and test selection
########################################################

# datdir = "/glade/derecho/scratch/ruppert/piccolo/"
datdir = "/glade/campaign/univ/uokl0053/"
# datdir = "/ourdisk/hpc/radclouds/auto_archive_notyet/tape_2copies/piccolo/"

case = "sept1-4"
test_process = "ctl"
wrf_dom = "wrf_fine"
nmem = 5 # number of ensemble members

########################################################
# New functions and single-use calls go here
########################################################

# Get variable lists
vars2d = var_list_2d()
# vars3d = var_list_3d()

# Ens-member string tags (e.g., memb_01, memb_02, etc.)
memb0=1 # Starting member to read
memb_nums_str=np.arange(memb0,nmem+memb0,1).astype(str)
nustr = np.char.zfill(memb_nums_str, 2)
memb_all=np.char.add('memb_',nustr)

def memb_dir_settings(memb_dir):
    wrfdir = datdir+case+'/'+memb_dir+'/'+test_process+"/"+wrf_dom+"/"
    outdir = wrfdir+"post_proc/"
    os.makedirs(outdir, exist_ok=True)
    # Get WRF file list, dimensions
    wrffiles = get_wrf_file_list(wrfdir, "wrfout_d01*")
    # hffiles = get_wrf_file_list(wrfdir, "hfout_d01*")
    lat, lon, nx1, nx2, nz, npd = wrf_dims(wrffiles[0])
    nfiles = len(wrffiles)
    # New vertical dimension for pressure levels
    # dp = 25 # hPa
    # pres = np.arange(1000, 25, -dp)
    # nznew = len(pres)
    return outdir, wrffiles, nfiles, npd

In [14]:
# Read and reduce variables for a given time step
def get_2d_special_vars_itimestep(ds, it_file):
    qv = getvar(ds, "QVAPOR", timeidx=it_file)#, cache=cache)
    dp = get_dp(ds, timeidx=it_file) # Pa
    # pclass
    ipclass = wrf_pclass(ds, dp, it_file)#ALL_TIMES)
    # pw
    ipw = vert_int(qv, dp)
    # pw_sat
    # qvsat = get_rv_sat(ds, pwrf, it_file)#ALL_TIMES)
    # tmpk = getvar(ds, 'tk', timeidx=it_file) # K
    # qvsat = rv_saturation(tmpk.values, pwrf.values) # kg/kg
    # qvsat = xr.DataArray(qvsat, coords=qv.coords, dims=qv.dims, attrs=qv.attrs)
    # ipw_sat = vert_int(qvsat, dp)
    # vertical mass flux
    wa = getvar(ds, "wa", timeidx=it_file)
    ivmf = vert_int(wa, dp)
    return ipclass, ipw, ivmf#, ipw_sat

# Loop over WRF input file time steps to read and reduce variables
def get_2d_special_vars_iwrf(file, var_list_special):
    ds = Dataset(file)
    # Loop over dataset time steps
    nt_file = ds.dimensions['Time'].size
    # for it_file in range(nt_file):
    for it_file in range(2):
        ipclass, ipw, ivmf = get_2d_special_vars_itimestep(ds, it_file)
        if it_file == 0:
            pclass_ifile = ipclass
            pw_ifile = ipw
            # pw_sat_ifile = ipw_sat
            vmf_ifile = ivmf
        else:
            pclass_ifile = xr.concat((pclass_ifile, ipclass), 'Time')
            pw_ifile = xr.concat((pw_ifile, ipw), 'Time')
            # pw_sat_ifile = xr.concat((pw_sat_ifile, ipw_sat), 'Time')
            vmf_ifile = xr.concat((vmf_ifile, ivmf), 'Time')
    ds.close()
    return pclass_ifile, pw_ifile, vmf_ifile#, pw_sat_ifile

In [17]:
if do_2d_special:

    var_list_special = ['pclass', 'pw', 'vmf']

    memb_dir = memb_all[0]

    # for memb_dir in memb_all:

    # memb_dir = memb_all[comm.rank]

    print("Processing "+memb_dir)
    outdir, wrffiles, nfiles, npd = memb_dir_settings(memb_dir)

    # Read in variable from WRF files
    for ifile in range(nfiles):
    # for ifile in range(1):

        # Open the WRF file
        wrffile = wrffiles[ifile]

        # Process variables
        pclass_ifile, pw_ifile, pw_sat_ifile = get_2d_special_vars_iwrf(wrffile, var_list_special)

        # Concatenate variables

        if ifile == 0:
            # pclass
            pclass_all = pclass_ifile
            # pw
            pw_all = pw_ifile
            # pw_sat
            pw_sat_all = pw_sat_ifile
        else:
            pclass_all = xr.concat((pclass_all, pclass_ifile), 'Time')
            pw_all = xr.concat((pw_all, pw_ifile), 'Time')
            pw_sat_all = xr.concat((pw_sat_all, pw_sat_ifile), 'Time')

    # Remove duplicate time steps
    pclass_all = pclass_all.drop_duplicates(dim="Time", keep='first')
    pw_all     = pw_all.drop_duplicates(dim="Time", keep='first')
    pw_sat_all = pw_sat_all.drop_duplicates(dim="Time", keep='first')

    # Write out the variables
    # var_name='pclass'
    # write_ncfile(outdir, pclass_all, var_name)
    # var_name='pw'
    # write_ncfile(outdir, pw_all, var_name)
    # var_name='pw_sat'
    # write_ncfile(outdir, pw_sat_all, var_name)

print("Done writing out special 2D variables")

Processing member memb_01


: 